## Testing Geoapify Places API

In [8]:
import sys
import pandas as pd
import numpy as np
# change working directory to the root of the project
sys.path.append('../')
from src.geoapify_api import geoapify_pipeline
from src.methods.data_ingestion import convert_easting_northing_to_long_lat
import importlib

#### Postcode input to be converted to longitude and latitude.

Postcode data was downloaded from: https://osdatahub.os.uk/downloads/open/CodePointOpen

In [9]:
postcode_path = '../data/postcode_data/Data/'
# create function for getting postcode data
def get_postcode_data(input_postcode: str, postcode_path) -> pd.DataFrame:
    """Get postcode data from postcode csv files"""
    # Check if postcode starts with two or one letters
    if input_postcode[:2].isalpha() == True:
        # match first two letters of postcode to file
        postcode_file = postcode_path + input_postcode[:2] + '.csv'
    else:
        # match first letter of postcode to file
        postcode_file = postcode_path + input_postcode[0] + '.csv'
    # read in file
    postcode_df = pd.read_csv(postcode_file, names=['postcode', 'pqi', 'easting', 'northing'], header=None, usecols=[0, 1, 2, 3])
    #filter to input postcode
    postcode_df = postcode_df[postcode_df['postcode'] == input_postcode]
    # convert easting and northing to longitude and latitude
    postcode_df['longitude'], postcode_df['latitude'] = convert_easting_northing_to_long_lat(postcode_df['easting'].iloc[0], postcode_df['northing'].iloc[0])
    return postcode_df

# test function
postcode = 'OX1 1AA'
post_df = get_postcode_data(postcode, postcode_path)
post_df

,postcode,pqi,easting,northing,longitude,latitude
0,OX1 1AA,10,450673,205980,51.750236,-1.267349


In [10]:
importlib.reload(sys.modules['src.geoapify_api'])
lat = post_df['latitude'].iloc[0]
long = post_df['longitude'].iloc[0]
request_params = {
    "apiKey": "493bbfea1f124b27906cf052aa330680",
    "categories": "tourism, leisure, entertainment",
    "limit": "500",
}
filters = {
        "circle": f"{lat},{long},5000",
}

url = 'https://api.geoapify.com/v2/places?'

df, res = geoapify_pipeline(url, request_params, filters)

c:\Users\nedee\Python_Projects\HolidayPlanner\workflows\..\src\geoapify_api.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[col] = df[col]
c:\Users\nedee\Python_Projects\HolidayPlanner\workflows\..\src\geoapify_api.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[col] = df[col]


In [11]:
from src.helpers.time_tracker import timings_to_dataframe
timings_to_dataframe()

,Function,Time
0,convert_easting_northing_to_long_lat,0.915466
1,create_url,0.000026
2,call_api,10.045113
3,format_to_df,1.356553
4,get_unique_words,0.004916
5,add_weather_tags,0.007419
6,geoapify_pipeline,11.510677


In [12]:
# In category column filter to look for ['tourism,'theatre'] and remove ['artwork','memorial']
df1 = df[df['category_keywords'].apply(lambda x: 'tourism' in x or 'theatre' in x)]
df1 = df1[df1['category_keywords'].apply(lambda x: 'artwork' not in x and 'memorial' not in x)]
# remove rows containing either only 'sights' or only ['sights','tourism'] in either order
df1 = df1[~df1['category_keywords'].apply(lambda x: x == ['sights'] or x == ['sights','tourism'] or x == ['tourism','sights'])]
df1

,name,country,country_code,state,county,city,postcode,district,suburb,street,...,state_code,formatted,address_line1,address_line2,quarter,license,opening_hours,website,category_keywords,weather_tags
0,Ashmolean Museum,United Kingdom,gb,England,Oxfordshire,Oxford,OX1 2PH,St Thomas',City Centre,Beaumont Street,...,ENG,"Ashmolean Museum, Beaumont Street, Oxford, OX1...",Ashmolean Museum,"Beaumont Street, Oxford, OX1 2PH, United Kingdom",NaN,Open Database Licence,Tu-Su 10:00-17:00; Fr[-1] 10:00-20:00,https://www.ashmolean.org/,"[no_fee, no, tourism, museum, entertainment, b...",1
2,Sheldonian Theatre,United Kingdom,gb,England,Oxfordshire,Oxford,OX1 3AZ,Holywell,City Centre,Broad Street,...,ENG,"Sheldonian Theatre, Broad Street, Oxford, OX1 ...",Sheldonian Theatre,"Broad Street, Oxford, OX1 3AZ, United Kingdom",NaN,Open Database Licence,"Mo-Sa 10:00-12:30, 14:00-16:30",https://www.sheldonian.ox.ac.uk/,"[culture, university, tourism, theatre, attrac...",1
3,Museum of the History of Science,United Kingdom,gb,England,Oxfordshire,Oxford,OX1 3BQ,Holywell,City Centre,Broad Street,...,ENG,"Museum of the History of Science, Broad Street...",Museum of the History of Science,"Broad Street, Oxford, OX1 3BQ, United Kingdom",NaN,Open Database Licence,Tu-Su 12:00-17:00,https://hsm.ox.ac.uk/,"[no_fee, no, tourism, limited, museum, enterta...",1
4,University of Oxford Botanic Garden,United Kingdom,gb,England,Oxfordshire,Oxford,OX1 4AZ,Holywell,City Centre,Rose Lane,...,ENG,"University of Oxford Botanic Garden, Rose Lane...",University of Oxford Botanic Garden,"Rose Lane, Oxford, OX1 4AZ, United Kingdom",NaN,Open Database Licence,NaN,https://www.obga.ox.ac.uk/,"[access, fee, yes, tourism, leisure, park, att...",5
5,Worcester College,United Kingdom,gb,England,Oxfordshire,Oxford,OX1 2HB,St Thomas',City Centre,Walton Street,...,ENG,"Worcester College, Walton Street, Oxford, OX1 ...",Worcester College,"Walton Street, Oxford, OX1 2HB, United Kingdom",NaN,Open Database Licence,Mo-Su 14:00-16:00,https://www.worc.ox.ac.uk,"[university, tourism, attraction, education]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,Tolkien's Table,United Kingdom,gb,England,Oxfordshire,Oxford,OX1 4DX,Holywell,City Centre,Deadman's Walk,...,ENG,"Tolkien's Table, Deadman's Walk, Oxford, OX1 4...",Tolkien's Table,"Deadman's Walk, Oxford, OX1 4DX, United Kingdom",NaN,Open Database Licence,NaN,NaN,"[tourism, attraction]",3
358,Michael Pilch Studio,United Kingdom,gb,England,Oxfordshire,Oxford,OX1 3TD,Holywell,City Centre,Jowett Walk,...,ENG,"Michael Pilch Studio, Jowett Walk, Oxford, OX1...",Michael Pilch Studio,"Jowett Walk, Oxford, OX1 3TD, United Kingdom",NaN,Open Database Licence,NaN,NaN,"[theatre, culture, entertainment]",1
361,Muntjac Point,United Kingdom,gb,England,Oxfordshire,Oxford,OX1 4UW,NaN,Iffley,River Walk,...,ENG,"Muntjac Point, River Walk, Oxford, OX1 4UW, Un...",Muntjac Point,"River Walk, Oxford, OX1 4UW, United Kingdom",NaN,Open Database Licence,NaN,NaN,"[viewpoint, tourism, attraction]",5
392,Tank Bridge,United Kingdom,gb,England,Oxfordshire,Oxford,OX2 0NJ,NaN,Medley,Bailey Bridge,...,ENG,"Tank Bridge, Bailey Bridge, Oxford, OX2 0NJ, U...",Tank Bridge,"Bailey Bridge, Oxford, OX2 0NJ, United Kingdom",NaN,Open Database Licence,NaN,NaN,"[viewpoint, tourism, attraction]",5
